# Reglas de asociación

### Librerías a utilizar:

- ggpubr
- arulesViz

### ¿Qué son las reglas de asociación?

Las reglas de asociación es una herramienta que goza de gran popularidad para la minería de datos en bases de datos, es una técnica de aprendizaje **no supervisado**. La idea es encontrar valores conjuntos para una serie de características $X = (X_1, X_2, X_3, ..., X_p)$ , la meta es encontrar las más frecuentes. Generalmente se utiliza en en datos con representación binaria (1=está y 0= no está) $X_j \in \{0,1\}$, a esto se le suele llamar *market basket* (canasta de mercado), dado que el problema nace desde el problema de encontrar patrones en bases de datos comerciales. 



### ¿Cuándo las utilizo?
- Cuando quiero encontrar un comportamiento repetitivo en base a la asociación de las características del estudio.
- Descubrir un conjunto de hechos en común.
- Se quiere conocer relaciones entre las variables.

### Historia:

> Basado en el concepto de regla fuerte, Agrawal et al.3​ presentaron un trabajo en el que indicaban las reglas de asociación que descubrían las relaciones entre los datos recopilados a gran escala en los sistemas de terminales de punto de venta de unos supermercados (Wikipedia).

Un ejemplo de regla sería:

$$ cebollas, vegetales => carne$$

Con este tipo de regla es posible conocer como se comporta el mercado, por ejemplo podría poner en pasillos cercanos los productos $carne, vegetales$ y $carne$.

### Conceptos very importantes

- Antecedente: Es la parte izquierda de la regla. Indica que relaciones provocan algo.
- Consecuente: Es la parte derecha. Indica la conclusión de la regla.
- Soporte: Es una medida de calidad de la regla. Para una regla $A => B$, el soporte es la fracción de observaciones en la unión del antecedente y consecuente.
$$ sop(X) = \frac{X}{D}$$
Donde $X$ corresponde a las transacciones y $D$ a la base de datos.

- Confianza: Corresponde a otra medida de interés de la regla. Se define como el soporte de la regla dividodo por el soporte del antecedente.
$$ conf(A=>B) = \frac{sop(A => B)}{sop(A)}$$

** Algo así como la probabildiad condicional $P(B|A)$.


- *Lift*: expresa cuál es la proporción del soporte observado de un conjunto de productos respecto del soporte teórico de ese conjunto dado el supuesto de independencia.

    - lift == 1 ? El conjunto aparece una cantidad de veces acorde a lo esperado bajo condiciones de independencia.
    - lift > 1 ? Ese conjunto aparece una cantidad de veces superior a lo esperado bajo condiciones de independencia (por lo que se puede intuir que existe una relación que hace que los productos se encuentren en el conjunto más veces de lo normal)
    - lift < 1 ? Ese conjunto aparece una cantidad de veces inferior a lo esperado bajo condiciones de independencia (por lo que se puede intuir que existe una relación que hace que los productos no estén formando parte del mismo conjunto más veces de lo normal)

$$ lift(A => B) = \frac{conf(A=>B)}{sop(B)}$$

** mide si la regla se debió al azar

> Cuanto más se aleje el valor de lift de 1, más evidencias de que la regla no se debe a un artefacto aleatorio, es decir, mayor la evidencia de que la regla representa un patrón real ([Joaquin_AR](https://rpubs.com/Joaquin_AR/397172))

### Ejemplito textual:

- Se tiene un conjunto $K = \{mantequilla \quad de \quad maní, jalea, pan\}$
- Se tiene una regla *mantequilla de maní*, *jalea* => *pan*
- El soporte de la regla es de 0.03 -> esto significa que los elementos aparecen juntos un 3% en la canasta.
- La confianza de la regla es de 0.82 -> esto significa que el 82% del tiempo que la mantequilla de maní y jalea fueron comprados, también se compró pan.

- Si el pan apareció un 43% del total de canastas, entonces la regla tendría un *lift* de 1.95 ¿qué significa esto?



In [ ]:
library("ggpubr")
library("cowplot")

### Ejemplo: 
Para el conjunto de datos de [semillas de trigo](https://archive.ics.uci.edu/ml/datasets/seeds) se quiere estudiar reglas de interés que tengan como consecuente la clase.

Recordar que el conjunto de datos a explicar está completamente balanceado (misma cantidad de observaciones por clase) y se tienen 7 características continuas (todas positivas).

In [ ]:
columns = c("area", "perimeter", "compactness", "length", "width", "AC", "lengthGroove", "class")
url = "https://www.dl.dropboxusercontent.com/s/wrexlo5im3g5ioi/seeds_dataset.csv"
seeds = read.csv(url, header = F, sep=",", col.names = columns)
seeds$class = factor(seeds$class, levels = c(1,2,3), labels = c("Kama", "Rosa", "Canadian"))

In [ ]:
head(seeds)

In [ ]:
summary(seeds[seeds$class == "Kama",])

In [ ]:
summary(seeds[seeds$class == "Rosa",])

In [ ]:
summary(seeds[seeds$class == "Canadian",])


Al igual que en las experiencias pasadas, para recordar cómo se encuentran las clases en función de sus características, se tiene que:

In [ ]:
boxplot.area =  ggboxplot(data = seeds, x = "class", y = "area", color = "class", add = "jitter") + border() 
ydens = axis_canvas(boxplot.area, axis = "y", coord_flip = TRUE) + geom_density(data = seeds, aes(x = area, fill = class), alpha = 0.7, size = 0.2) + coord_flip()
boxplot.area = insert_yaxis_grob(boxplot.area, ydens, grid::unit(.2, "null"), position = "right")
ggdraw(boxplot.area)

# perimeter
boxplot.perimeter =  ggboxplot(data = seeds, x = "class", y = "perimeter", color = "class", add= "jitter") + border()
ydens = axis_canvas(boxplot.perimeter, axis = "y", coord_flip = TRUE) + geom_density(data = seeds, aes(x = perimeter, fill = class), alpha = 0.7, size = 0.2) + coord_flip()
boxplot.perimeter = insert_yaxis_grob(boxplot.perimeter, ydens, grid::unit(.2, "null"), position = "right")
ggdraw(boxplot.perimeter)

# compactness
boxplot.compactness =  ggboxplot(data = seeds, x = "class", y = "compactness", color = "class") + border()
ydens = axis_canvas(boxplot.compactness, axis = "y", coord_flip = TRUE) + geom_density(data = seeds, aes(x = compactness, fill = class), alpha = 0.7, size = 0.2) + coord_flip()
boxplot.compactness = insert_yaxis_grob(boxplot.compactness, ydens, grid::unit(.2, "null"), position = "right")
ggdraw(boxplot.compactness)

# length
boxplot.length =  ggboxplot(data = seeds, x = "class", y = "length", color = "class") + border()
ydens = axis_canvas(boxplot.length, axis = "y", coord_flip = TRUE) + geom_density(data = seeds, aes(x = length, fill = class), alpha = 0.7, size = 0.2) + coord_flip()
boxplot.length = insert_yaxis_grob(boxplot.length, ydens, grid::unit(.2, "null"), position = "right")
ggdraw(boxplot.length)

# width
boxplot.width =  ggboxplot(data = seeds, x = "class", y = "width", color = "class") + border()
ydens = axis_canvas(boxplot.width, axis = "y", coord_flip = TRUE) + geom_density(data = seeds, aes(x = width, fill = class), alpha = 0.7, size = 0.2) + coord_flip()
boxplot.width = insert_yaxis_grob(boxplot.width, ydens, grid::unit(.2, "null"), position = "right")
ggdraw(boxplot.width)

# AC
boxplot.AC =  ggboxplot(data = seeds, x = "class", y = "AC", color = "class") + border()
ydens = axis_canvas(boxplot.AC, axis = "y", coord_flip = TRUE) + geom_density(data = seeds, aes(x = AC, fill = class), alpha = 0.7, size = 0.2) + coord_flip()
boxplot.AC = insert_yaxis_grob(boxplot.AC, ydens, grid::unit(.2, "null"), position = "right")
ggdraw(boxplot.AC)

# lengthGroove
boxplot.lengthGroove =  ggboxplot(data = seeds, x = "class", y = "lengthGroove", color = "class") + border()
ydens = axis_canvas(boxplot.lengthGroove, axis = "y", coord_flip = TRUE) + geom_density(data = seeds, aes(x = lengthGroove, fill = class), alpha = 0.7, size = 0.2) + coord_flip()
boxplot.lengthGroove = insert_yaxis_grob(boxplot.lengthGroove, ydens, grid::unit(.2, "null"), position = "right")
ggdraw(boxplot.lengthGroove)

Se logra apreciar que algunas características como el área y perímetro parecen ser un buen discriminador en sus respectivos gráficos. Si vemos como se ve distribuida la clase en estas dos "dimensiones" existe una tendencia. ¿A qué se parece esto? (spoiler: clustering)

In [ ]:
ggscatter(data = seeds, x = "area", y = "perimeter", color = "class")

## Reglas de asosciación en R
Al observar el gráfico anterior podríamos decir a **priori** que la clase Canadian tiene un área pequeña y perímtro pequeño.

Dejando de lado ese precario análisis exploratorio (solo repaso), es hora de ver como se ven las reglas de asosiación en R 😬.

In [ ]:
library("arulesViz")

Primero se debe discretizar los datos a trabajar (para esto deben ver la naturaleza de su problema), en este caso discretizaremos (arbitrariamente) en dos rangos por característica.

In [ ]:
seeds.rules = seeds

area = c(-Inf, 14.85, Inf)
area.names = c("Pequeña", "Grande")

perimeter = c(-Inf, 14.56, Inf)
perimeter.names = c("Pequeña", "Grande")

compactness = c(-Inf, 0.8710, Inf)
compactness.names = c("Pequeña", "Grande")

lenght = c(-Inf, 5.629, Inf)
lenght.names = c("Pequeña", "Grande")

width = c(-Inf, 3.259, Inf)
width.names = c("Pequeña", "Grande")

ac = c(-Inf, 3.7002, Inf)
ac.names = c("Pequeña", "Grande")

lg = c(-Inf, 5.408, Inf)
lg.names = c("Pequeña", "Grande")


seeds.rules$area = cut(seeds.rules$area, breaks = area, labels = area.names)
seeds.rules$perimeter = cut(seeds.rules$perimeter, breaks = perimeter, labels = perimeter.names)
seeds.rules$compactness = cut(seeds.rules$compactness, breaks = compactness, labels = compactness.names)
seeds.rules$length = cut(seeds.rules$length, breaks = lenght, labels = lenght.names)
seeds.rules$width = cut(seeds.rules$width, breaks = width, labels = width.names)
seeds.rules$AC = cut(seeds.rules$AC, breaks = ac, labels = ac.names)
seeds.rules$lengthGroove = cut(seeds.rules$lengthGroove, breaks = lg, labels = lg.names)

In [ ]:
head(seeds.rules)

Recordemos el gráfico de dispersión de [área vs perímetro](#Reglas-de-asosciación-en-R), en donde la premisa era que la clase Canadian tenían un área pequeña y perímetro pequeño. Busquemos este antecedente en el conjunto de datos...

In [ ]:
seeds.rules[seeds.rules$area == "Pequeña" &
            seeds.rules$perimeter == "Pequeña",]

Como se logra apreciar, esa premisa resultaba bien solo para el gráfico, pero en realidad para buscar instancias de tipo Canadian existen **reglas** más complejas y necesitamos buscarlas. Para eso utilizamos el algoritmo **apriori**.

In [ ]:
rules = apriori(
    data = seeds.rules, 
    parameter=list(support = 0.2, minlen = 2, maxlen = 6, target="rules")    
)


inspect(sort(x = rules, decreasing = TRUE, by = "confidence"))

## ¿Qué reglas nos interesan para nuestro conjunto de datos?

Primero hay que preguntarse si las reglas anteriores son de interés para mi estudio.

¿me sirve decir que *{compactness=Pequeña ^ class=Canadian} => {width=Pequeña}*?

Filtrarenos para los consecuentes que contemplen la clase...

In [ ]:
rules = apriori(
    data = seeds.rules, 
    parameter=list(support = 0.2, minlen = 2, maxlen = 6, target="rules"),
    appearance=list(rhs = c("class=Canadian", "class=Kama", "class=Rosa"))
)

inspect(sort(x = rules, decreasing = TRUE, by = "confidence"))

¿Qué nos dice esto?
Una serie de reglas para decir si una semilla es de tipo Canadian y nos entrega unas métricas de calidad de las reglas, como:

- Support
- Lift
- Confidence
- Coverage

```R
      lhs                       rhs                support confidence  coverage     lift count
[1]   {compactness=Pequeña,                                                                   
       length=Pequeña,                                                                        
       AC=Grande}            => {class=Canadian} 0,2380952  0,9615385 0,2476190 2,884615    50
[2]   {compactness=Pequeña,                                                                   
       length=Pequeña,                                                                        
       width=Pequeña,                                                                         
       AC=Grande}            => {class=Canadian} 0,2380952  0,9615385 0,2476190 2,884615    50
[3]   {area=Pequeña,                                                                          
       compactness=Pequeña,                                                                   
       length=Pequeña,                                                                        
       AC=Grande}            => {class=Canadian} 0,2380952  0,9615385 0,2476190 2,884615    50
[4]   {perimeter=Pequeña,                                                                     
       compactness=Pequeña,                                                                   
       length=Pequeña,                                                                        
       AC=Grande}            => {class=Canadian} 0,2380952  0,9615385 0,2476190 2,884615    50
[5]   {area=Pequeña,                                                                          
       compactness=Pequeña,                                                                   
       length=Pequeña,                                                                        
       width=Pequeña,                                                                         
       AC=Grande}            => {class=Canadian} 0,2380952  0,9615385 0,2476190 2,884615    50
[6]   {perimeter=Pequeña,                                                                     
       compactness=Pequeña,                                                                   
       length=Pequeña,                                                                        
       width=Pequeña,                                                                         
       AC=Grande}            => {class=Canadian} 0,2380952  0,9615385 0,2476190 2,884615    50
[7]   {area=Pequeña,                                                                          
       perimeter=Pequeña,                                                                     
       compactness=Pequeña,                                                                   
       length=Pequeña,                                                                        
       AC=Grande}            => {class=Canadian} 0,2380952  0,9615385 0,2476190 2,884615    50
[8]   {compactness=Pequeña,                                                                   
       length=Pequeña,                                                                        
       AC=Grande,                                                                             
       lengthGroove=Pequeña} => {class=Canadian} 0,2333333  0,9607843 0,2428571 2,882353    49
[9]   {compactness=Pequeña,                                                                   
       length=Pequeña,                                                                        
       width=Pequeña,                                                                         
       AC=Grande,                                                                             
       lengthGroove=Pequeña} => {class=Canadian} 0,2333333  0,9607843 0,2428571 2,882353    49
[10]  {area=Pequeña,                                                                          
       compactness=Pequeña,                                                                   
       length=Pequeña,                                                                        
       AC=Grande,                                                                             
       lengthGroove=Pequeña} => {class=Canadian} 0,2333333  0,9607843 0,2428571 2,882353    49


```

Busquemos una regla en el conjunto de datos:

In [ ]:
seeds.rules[seeds.rules$area == "Pequeña" &
            seeds.rules$compactness == "Pequeña" &
            seeds.rules$length == "Pequeña" &
            seeds.rules$AC == "Grande" &
            seeds.rules$lengthGroove == "Pequeña",]

In [ ]:
seeds.rules[seeds.rules$compactness == "Pequeña" &
            seeds.rules$length == "Pequeña" &
            seeds.rules$AC == "Grande",]